# Fine-Tuning LLMs

## 1. Set up Dependencies and Load Dataset and LLM

In [1]:
!pip install datasets evaluate rouge_score peft -q

  Preparing metadata (setup.py) ... done


In [2]:
import torch
import time
import evaluate
import pandas as pd
import numpy as np

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer
from datasets import load_dataset

In [3]:
dataset = load_dataset('knkarthick/dialogsum')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Load the pre-trained [Flan-T5 model](https://huggingface.co/docs/transformers/model_doc/flan-t5) and its tokenizer from HuggingFace. Notice that you will be using the [small version](https://huggingface.co/google/flan-t5-base) of Flan-T5. Setting `torch_dtype=torch.bfloat16` specifies the data type to be used by this model, which can reduce GPU memory usage since `bfloat16` uses half as much memory per number compared to `float32`, the default precision for most models.

In [4]:
model_name = 'google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)#, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

## 2. Test the Model with Zero-Shot Inferencing

Test the model with zero-shot inference.

In [5]:
index = 42
dash_line = '-' * 100

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:\n"
inputs = tokenizer(prompt, return_tensors='pt')
output = original_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
original_model_summary = tokenizer.decode(output, skip_special_tokens=True)

print(dash_line)
print(f'INPUT PROMPT:\n{dialogue}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{original_model_summary}\n')

----------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: I don't know how to adjust my life. Would you give me a piece of advice?
#Person2#: You look a bit pale, don't you?
#Person1#: Yes, I can't sleep well every night.
#Person2#: You should get plenty of sleep.
#Person1#: I drink a lot of wine.
#Person2#: If I were you, I wouldn't drink too much.
#Person1#: I often feel so tired.
#Person2#: You better do some exercise every morning.
#Person1#: I sometimes find the shadow of death in front of me.
#Person2#: Why do you worry about your future? You're very young, and you'll make great contribution to the world. I hope you take my advice.
----------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# wants to adjust #Person1#'s life and #Person2# suggests #Person1# be positive and stay healthy.
-------------------------------------------------------

You can see that the model struggles to summarize the dialogue compared to the baseline summary, and simply repeats the first sentence from the dialogue.

## 3. Perform Full Fine-Tuning

### 3.1 Preprocess the Dataset

You need to convert the dialog-summary (prompt-response) pairs into explicit instructions for the LLM. Prepend an instruction to the start of the dialog with `Summarize the following conversation.`, and to the start of the summary with `Summary:` as follows:

Training prompt (dialogue):
```
Summarize the following conversation.
Alice: This is her part of the conversation.
Bob: This is his part of the conversation.    
Summary:
```

Training response (summary):
```
Both Alice and Bob participated in the conversation.
```

**Exercise**: Write a function to tokenize a batch of examples from the dialogue dataset. The function should concatentate the dialogues with the predefined prompt, tokenize them along with their summaries, and define the tokenized summaries as the labels.

In [6]:
def tokenize(examples):
    ### WRITE YOUR CODE HERE

    # Prepend the instruction to the dialogues and append 'Summary:' to indicate where the model should generate a summary.
    prompts = ["Summarize the following conversation.\n" + dialogue for dialogue in examples['dialogue']]
    summaries = ["Summary:\n" + summary for summary in examples['summary']]

    # Tokenize both the prompts and the summaries.
    # The prompts are treated as the input to the model, and the summaries as what the model should generate.
    model_inputs = tokenizer(prompts, padding="max_length", truncation=True, max_length=512)
    # Use tokenizer's batch_encode_plus for the summaries to get their token IDs but ignore the attention masks.
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(summaries, padding="max_length", truncation=True, max_length=128)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [7]:
tokenized_dataset = dataset.map(tokenize, batched=True)

# Apply the tokenization function to the entire dataset.
# tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=dataset['test'].column_names)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

### 3.2 Fine-Tune the Model

**Exercise**: Utilize the Hugging Face Trainer API for training the model on the preprocessed dataset. Define the training arguments, a data collator, and create a `Seq2SeqTrainer` instance. Train the model for one epoch.

In [8]:
### WRITE YOUR CODE HERE

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

# Define the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    #fp16=True, # not compabible with bfloat16, enable for AMP
)

# Initialize the data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=original_model)

# Initialize Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Training a fully fine-tuned version of the model should take about 10 minutes on a Google Colab GPU machine.

In [9]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.394700,0.333909


TrainOutput(global_step=1558, training_loss=1.1975039456407892, metrics={'train_runtime': 2305.068, 'train_samples_per_second': 5.405, 'train_steps_per_second': 0.676, 'total_flos': 8532076611502080.0, 'train_loss': 1.1975039456407892, 'epoch': 1.0})

Save the model to a local folder:

In [10]:
model_path = './flan-t5-base-dialogsum-checkpoint'

original_model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('./flan-t5-base-dialogsum-checkpoint/tokenizer_config.json',
 './flan-t5-base-dialogsum-checkpoint/special_tokens_map.json',
 './flan-t5-base-dialogsum-checkpoint/spiece.model',
 './flan-t5-base-dialogsum-checkpoint/added_tokens.json',
 './flan-t5-base-dialogsum-checkpoint/tokenizer.json')

Create an instance of the `AutoModelForSeq2SeqLM` class for the instruct model:

Reload the original Flan-T5-base model:

In [11]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained('./flan-t5-base-dialogsum-checkpoint',
                                                       torch_dtype=torch.float16)

In [12]:
original_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype=torch.bfloat16)

### 3.3 Evaluate the Model Qualitatively (Human Evaluation)

**Exercise**: Make inferences for the same example as in Section 2, using the original model and the fully fine-tuned model.

In [13]:
### WRITE YOUR CODE HERE

# Assuming you have the dataset loaded and the tokenizer initialized
index = 42  # The example index as mentioned

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:"

# Generate summary with the original model
inputs = tokenizer(prompt, return_tensors='pt')
original_output = original_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
original_summary = tokenizer.decode(original_output, skip_special_tokens=True)

# Generate summary with the fine-tuned model
ft_inputs = tokenizer(prompt, return_tensors='pt')
ft_output = instruct_model.generate(ft_inputs['input_ids'], max_new_tokens=50)[0]
ft_summary = tokenizer.decode(ft_output, skip_special_tokens=True)

# Print summaries for comparison
# Print the dialogues, baseline human summary, and model-generated summaries for comparison
print(dash_line)
print(f'INPUT DIALOGUE:\n{dialogue}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'ORIGINAL MODEL GENERATION - ZERO SHOT:\n{original_summary}')
print(dash_line)
print(f'FINE-TUNED MODEL GENERATION:\n{ft_summary}\n')
print(dash_line)

----------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: I don't know how to adjust my life. Would you give me a piece of advice?
#Person2#: You look a bit pale, don't you?
#Person1#: Yes, I can't sleep well every night.
#Person2#: You should get plenty of sleep.
#Person1#: I drink a lot of wine.
#Person2#: If I were you, I wouldn't drink too much.
#Person1#: I often feel so tired.
#Person2#: You better do some exercise every morning.
#Person1#: I sometimes find the shadow of death in front of me.
#Person2#: Why do you worry about your future? You're very young, and you'll make great contribution to the world. I hope you take my advice.
----------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# wants to adjust #Person1#'s life and #Person2# suggests #Person1# be positive and stay healthy.
-----------------------------------------------------

The fine-tuned model is able to create a much better summary of the dialogue compared to the original model.

### 3.4 Evaluate the Model Quantitatively (with ROUGE Metric)

The [ROUGE metric](https://en.wikipedia.org/wiki/ROUGE_(metric)) helps quantify the validity of summarizations produced by models. It compares summarizations to a "baseline" summary which is usually created by a human. While not perfect, it does indicate the overall increase in summarization effectiveness that we have accomplished by fine-tuning.

In [14]:
rouge = evaluate.load('rouge')

**Exercise**: Generate the outputs for a sample of the test set with the fine-tuned model (use only the first 10 dialogues and summaries to save time).

In [15]:
### WRITE YOUR CODE HERE

# Generate outputs for the first 10 test dialogues
test_sample = dataset['test'].select(range(10))  # Selecting the first 10 examples for quick evaluation

original_model_summaries = []
instruct_model_summaries = []
human_baseline_summaries = [example['summary'] for example in test_sample]

for example in test_sample:
    prompt = f"Summarize the following conversation.\n{example['dialogue']}\nSummary:\n"
    inputs = tokenizer(prompt, return_tensors='pt')

    # Generate summary with the original model
    original_output = original_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
    original_model_summaries.append(tokenizer.decode(original_output, skip_special_tokens=True))

    # Generate summary with the fine-tuned model
    ft_output = instruct_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
    instruct_model_summaries.append(tokenizer.decode(ft_output, skip_special_tokens=True))


Evaluate the models computing ROUGE metrics:

In [16]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)]
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)]
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': 0.2231374643874644, 'rouge2': 0.09872463768115944, 'rougeL': 0.20800462962962962, 'rougeLsum': 0.21222150997150996}
INSTRUCT MODEL:
{'rouge1': 0.366502317548411, 'rouge2': 0.10567735312416163, 'rougeL': 0.27659827371232243, 'rougeLsum': 0.27723664051201646}


The results show substantial improvement in all ROUGE metrics:

In [17]:
print("Absolute percentage improvement of the instruct model over the original model:")

for key in instruct_model_results:
    improvement = instruct_model_results[key] - original_model_results[key]
    print(f'{key}: {improvement*100:.2f}%')

Absolute percentage improvement of the instruct model over the original model:
rouge1: 14.34%
rouge2: 0.70%
rougeL: 6.86%
rougeLsum: 6.50%


## 4. Perform Parameter Efficient Fine-Tuning (PEFT)

Now, let's perform **Parameter Efficient Fine-Tuning (PEFT)** instead of "full fine-tuning" as you did above. PEFT is a form of instruction fine-tuning that is much more efficient than full fine-tuning, with comparable evaluation results as you will see soon.

One of the most popular PEFT methods is **Low-Rank Adaptation (LoRA)**, which  introduces low-rank matrices to adapt the LLM with minimal additional parameters. In most cases, when someone says PEFT, they typically mean LoRA.  After fine-tuning for a specific task with LoRA, the result is that the original LLM remains unchanged and a newly-trained "LoRA adapter" emerges. This LoRA adapter is much smaller than the original LLM - on the order of a single-digit % of the original LLM size (MBs vs GBs).  

At inference time, the LoRA adapter is reunited and combined with its original LLM to serve the inference request. The benefit is that many LoRA adapters can re-use the original LLM which reduces overall memory requirements when serving multiple tasks and use cases.

### 4.1 Setup the LoRA model for Fine-Tuning

You first need to define the configuration of the LoRA model. Have a look at the configuration below. The key configuration element to adjust is the rank (`r`) of the adapter, which influences its capacity and complexity. Experiment with various ranks, such as 8, 16, or 32, and see how they affect the results.

In [26]:
with torch.no_grad():
    torch.cuda.empty_cache()

import gc

gc.collect() # Python thing

430

In [27]:
from peft import LoraConfig, TaskType, get_peft_model

lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=32,
    lora_alpha=32,
    lora_dropout=0.1
)

Add LoRA adapter layers/parameters to the original LLM to be trained:

In [28]:
peft_model = get_peft_model(original_model, lora_config)

The number of trainable model parameters in the LoRA model is:

In [29]:
peft_model.print_trainable_parameters()

trainable params: 3,538,944 || all params: 251,116,800 || trainable%: 1.4092820552029972


### 4.2 Train the LoRA Adapter

**Exercise**: Define training arguments and create a `Seq2SeqTrainer` instance for the LoRA model. Use a higher learning rate than full fine-tuning (e.g., `1e-3`).

In [31]:
### WRITE YOUR CODE HERE

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

# Define training arguments for LoRA model fine-tuning
training_args_peft = Seq2SeqTrainingArguments(
    output_dir="./peft_results",
    evaluation_strategy="epoch",
    learning_rate=1e-3,  # Higher learning rate for PEFT
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
)

# Initialize data collator
data_collator_peft = DataCollatorForSeq2Seq(tokenizer, model=peft_model)

# Initialize Seq2SeqTrainer for the LoRA model
peft_trainer = Seq2SeqTrainer(
    model=peft_model,
    args=training_args_peft,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    data_collator=data_collator_peft,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Train the PEFT adapter. Training should take about 6 minutes on a Google Colab GPU machine.

In [32]:
peft_trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.416500,0.343080


TrainOutput(global_step=1558, training_loss=0.7400074715179717, metrics={'train_runtime': 2130.6859, 'train_samples_per_second': 5.848, 'train_steps_per_second': 0.731, 'total_flos': 8667537195663360.0, 'train_loss': 0.7400074715179717, 'epoch': 1.0})

Save the model to a local folder:

In [33]:
peft_model.save_pretrained('./flan-t5-base-dialogsum-lora')

Load the PEFT model:

In [34]:
from peft import AutoPeftModelForSeq2SeqLM
from transformers import AutoTokenizer

peft_model = AutoModelForSeq2SeqLM.from_pretrained('./flan-t5-base-dialogsum-lora')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

Reload the original Flan-T5-base model:

In [35]:
original_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype=torch.bfloat16)

### 4.3 Evaluate the Model Qualitatively (Human Evaluation)

**Exercise**: Make inferences for the same example as in Sections 2 and 3, using the original model, the fully fine-tuned model and the PEFT model.

In [36]:
### WRITE YOUR CODE HERE

index = 42  # Example index for inference
dialogue = dataset['test'][index]['dialogue']  # Dialogue text for the example

prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:\n"  # Instruction prompt

# Tokenize the prompt for input to the models
inputs = tokenizer(prompt, return_tensors='pt')

# Generate summary with the original model
original_output = original_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
original_model_summary = tokenizer.decode(original_output, skip_special_tokens=True)

# Generate summary with the fully fine-tuned model
ft_output = instruct_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
ft_model_summary = tokenizer.decode(ft_output, skip_special_tokens=True)

# Generate summary with the PEFT model
peft_output = peft_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
peft_model_summary = tokenizer.decode(peft_output, skip_special_tokens=True)

# Print summaries for comparison
print(f'INPUT DIALOGUE:\n{dialogue}\n')
print(f'ORIGINAL MODEL SUMMARY:\n{original_model_summary}\n')
print(f'FULLY FINE-TUNED MODEL SUMMARY:\n{ft_model_summary}\n')
print(f'PEFT MODEL SUMMARY:\n{peft_model_summary}\n')

INPUT DIALOGUE:
#Person1#: I don't know how to adjust my life. Would you give me a piece of advice?
#Person2#: You look a bit pale, don't you?
#Person1#: Yes, I can't sleep well every night.
#Person2#: You should get plenty of sleep.
#Person1#: I drink a lot of wine.
#Person2#: If I were you, I wouldn't drink too much.
#Person1#: I often feel so tired.
#Person2#: You better do some exercise every morning.
#Person1#: I sometimes find the shadow of death in front of me.
#Person2#: Why do you worry about your future? You're very young, and you'll make great contribution to the world. I hope you take my advice.

ORIGINAL MODEL SUMMARY:
Person1: I'm not sure how to adjust my life.

FULLY FINE-TUNED MODEL SUMMARY:
Summary: #Person1# asks #Person2# to give #Person1# some advice. #Person2# advises #Person1# to get plenty of sleep and exercise.

PEFT MODEL SUMMARY:
Summary: #Person1# is pale and can't sleep well every night. #Person2# advises #Person1# to get plenty of sleep and exercise.



### 4.4 Evaluate the Model Quantitatively (with ROUGE Metric)

**Exercise**: Generate the outputs for a sample of the test set with the PEFT model (use only the first 10 dialogues and summaries to save time).

In [37]:
### WRITE YOUR CODE HERE

subset = dataset['test'].select(range(10))
human_baseline_summaries = [example['summary'] for example in subset]

original_model_summaries = []
ft_model_summaries = []
peft_model_summaries = []

for i in range(10):
    dialogue = dataset['test'][i]['dialogue']
    prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:\n"
    inputs = tokenizer(prompt, return_tensors='pt')

    # Original model
    original_output = original_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
    original_model_summaries.append(tokenizer.decode(original_output, skip_special_tokens=True))

    # Fully fine-tuned model
    ft_output = instruct_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
    ft_model_summaries.append(tokenizer.decode(ft_output, skip_special_tokens=True))

    # PEFT model
    peft_output = peft_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
    peft_model_summaries.append(tokenizer.decode(peft_output, skip_special_tokens=True))

Compute ROUGE score for this subset of the data.

In [38]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.2231374643874644, 'rouge2': 0.09872463768115944, 'rougeL': 0.20800462962962962, 'rougeLsum': 0.21222150997150996}
INSTRUCT MODEL:
{'rouge1': 0.366502317548411, 'rouge2': 0.10567735312416163, 'rougeL': 0.27659827371232243, 'rougeLsum': 0.27723664051201646}
PEFT MODEL:
{'rouge1': 0.37136357261357267, 'rouge2': 0.11454562882276613, 'rougeL': 0.2912397812397812, 'rougeLsum': 0.29363165363165367}


Notice, that PEFT model results are not too bad, while the training process was much easier!

Calculate the improvement of PEFT over the original model:

In [39]:
print("Absolute percentage improvement of the PEFT model over the original model:")

for key in peft_model_results:
    improvement = peft_model_results[key] - original_model_results[key]
    print(f'{key}: {improvement*100:.2f}%')

Absolute percentage improvement of the PEFT model over the original model:
rouge1: 14.82%
rouge2: 1.58%
rougeL: 8.32%
rougeLsum: 8.14%


Now calculate the improvement of PEFT over a full fine-tuned model:

In [40]:
print("Absolute percentage improvement of the PEFT model over the instruct model:")

for key in peft_model_results:
    improvement = peft_model_results[key] - instruct_model_results[key]
    print(f'{key}: {improvement*100:.2f}%')

Absolute percentage improvement of the PEFT model over the instruct model:
rouge1: 0.49%
rouge2: 0.89%
rougeL: 1.46%
rougeLsum: 1.64%


You can see a small percentage decrease in the ROUGE metrics vs. full fine-tuned. However, the training requires much less computing and memory resources.